# Introduction 

I am most interested in the work of the connectionists and neural networks. In this report, I will attempt to answer the following question:

Given an image of a Pokemon, can a convolutional neural network model be used to classify the Pokemon by type?

I had this inspiration from a dataset I recently found on Kaggle, shown here:

https://www.kaggle.com/vishalsubbiah/pokemon-images-and-types

## Background 

I refer the reader to the following Wikipedia article for more information on Pokemon:

https://en.wikipedia.org/wiki/Pok%C3%A9mon_(video_game_series)

For the purposes of context for this report, Pokemon is shorthand for Pocket Monsters, and it is a Japanese video game series developed for Nintendo gaming systems. In it, the player goes on an adventure where they assemble a team of 6 creatures, train them up to become strong, and compete for the recognition of becoming the most powerful trainer in the game world. Players in Pokemon compete by battling them against each other. As of this writing, there are 890 unique Pokemon. The Kaggle dataset mentioned above contains only 809 Pokemon, and was not updated for the additional 81 Pokemon introduced in Pokemon Sword and Shield in November 2019.

Each Pokemon has a primary and possibly a secondary type. For the purposes of complexity, we will only use the primary type of a Pokemon as a class label. There are 18 unique types of Pokemon, which each type having its own strengths and weaknesses in battle with respect to other types. Examples of types include Fire, Water, Grass, Ground, or Electric. We will investigate in this report the ability of a CNN to distinguish Pokemon by type based on their appearance in images.

# Implementation

First we load in the data:

In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
os.chdir('C:/Users/mkell/Downloads/pokemon-images-and-types')

In [3]:
pokemon=pd.read_csv('pokemon.csv')
print(np.unique(pokemon['Type1'], return_counts=True))
pokemon=pokemon.sort_values('Name')
pokemon=pokemon.reset_index(drop=True)
pokemon

(array(['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water'], dtype=object), array([ 72,  29,  27,  40,  18,  29,  53,   3,  27,  78,  32,  23, 105,
        34,  53,  46,  26, 114], dtype=int64))


,Name,Type1,Type2
0,abomasnow,Grass,Ice
1,abra,Psychic,NaN
2,absol,Dark,NaN
3,accelgor,Bug,NaN
4,aegislash-blade,Steel,Ghost
...,...,...,...
804,zoroark,Dark,NaN
805,zorua,Dark,NaN
806,zubat,Poison,Flying
807,zweilous,Dark,Dragon


This is the original dataset of 809 Pokemon. We will use the column 'Type1' for labeling. Next we load in the images:

In [4]:
images=np.empty((len(os.listdir('images/images')), 120, 120, 3))
count=0

for root, dirs, files in os.walk('images'):
    for i, file in enumerate(files):        
        path = os.path.join(root, file) 
        img=cv2.imread(path)        
        images[count] = img        
        count=count+1
        print("Loaded file "+str(count)+ " of "+str(len(os.listdir('images/images')))+ " ")              

Loaded file 1 of 809 
Loaded file 2 of 809 
Loaded file 3 of 809 
Loaded file 4 of 809 
Loaded file 5 of 809 
Loaded file 6 of 809 
Loaded file 7 of 809 
Loaded file 8 of 809 
Loaded file 9 of 809 
Loaded file 10 of 809 
Loaded file 11 of 809 
Loaded file 12 of 809 
Loaded file 13 of 809 
Loaded file 14 of 809 
Loaded file 15 of 809 
Loaded file 16 of 809 
Loaded file 17 of 809 
Loaded file 18 of 809 
Loaded file 19 of 809 
Loaded file 20 of 809 
Loaded file 21 of 809 
Loaded file 22 of 809 
Loaded file 23 of 809 
Loaded file 24 of 809 
Loaded file 25 of 809 
Loaded file 26 of 809 
Loaded file 27 of 809 
Loaded file 28 of 809 
Loaded file 29 of 809 
Loaded file 30 of 809 
Loaded file 31 of 809 
Loaded file 32 of 809 
Loaded file 33 of 809 
Loaded file 34 of 809 
Loaded file 35 of 809 
Loaded file 36 of 809 
Loaded file 37 of 809 
Loaded file 38 of 809 
Loaded file 39 of 809 
Loaded file 40 of 809 
Loaded file 41 of 809 
Loaded file 42 of 809 
Loaded file 43 of 809 
Loaded file 44 of 80

Loaded file 482 of 809 
Loaded file 483 of 809 
Loaded file 484 of 809 
Loaded file 485 of 809 
Loaded file 486 of 809 
Loaded file 487 of 809 
Loaded file 488 of 809 
Loaded file 489 of 809 
Loaded file 490 of 809 
Loaded file 491 of 809 
Loaded file 492 of 809 
Loaded file 493 of 809 
Loaded file 494 of 809 
Loaded file 495 of 809 
Loaded file 496 of 809 
Loaded file 497 of 809 
Loaded file 498 of 809 
Loaded file 499 of 809 
Loaded file 500 of 809 
Loaded file 501 of 809 
Loaded file 502 of 809 
Loaded file 503 of 809 
Loaded file 504 of 809 
Loaded file 505 of 809 
Loaded file 506 of 809 
Loaded file 507 of 809 
Loaded file 508 of 809 
Loaded file 509 of 809 
Loaded file 510 of 809 
Loaded file 511 of 809 
Loaded file 512 of 809 
Loaded file 513 of 809 
Loaded file 514 of 809 
Loaded file 515 of 809 
Loaded file 516 of 809 
Loaded file 517 of 809 
Loaded file 518 of 809 
Loaded file 519 of 809 
Loaded file 520 of 809 
Loaded file 521 of 809 
Loaded file 522 of 809 
Loaded file 523 

In [5]:
images.shape

(809, 120, 120, 3)

Some Pokemon types are intuitive and some are not. For example, the first and third Pokemon in the original dataset are Ice-type Pokemon, which is supported by their white, snowy appearance. However, the second Pokemon is Psychic-type, which is not immediately evident from its appearance. The difficulty of this task for humans is present because of this fact. Thus, we hope to see how difficult this task is for a neural network.

Note that the original dataset contains only 809 images in 18 classes. This is hardly enough data with which to train a model. Since there are no more official Pokemon to draw from, we can artificially enlarge our dataset by generating copies of the original images, done below:

In [6]:
os.chdir('C:/Users/mkell/Dropbox/Spring 2020/Artificial Intelligence/pokemon-type-classifier/pokemon-classifier')

Next we preprocess the data:

In [7]:
image_labels=np.array(pokemon['Type1'])
image_labels

array(['Grass', 'Psychic', 'Dark', 'Bug', 'Steel', 'Rock', 'Steel',
       'Normal', 'Psychic', 'Water', 'Dragon', 'Rock', 'Normal', 'Grass',
       'Electric', 'Rock', 'Water', 'Poison', 'Fire', 'Normal', 'Rock',
       'Rock', 'Bug', 'Rock', 'Fairy', 'Steel', 'Ice', 'Normal', 'Rock',
       'Ice', 'Dragon', 'Psychic', 'Water', 'Normal', 'Dragon', 'Ground',
       'Ghost', 'Rock', 'Water', 'Water', 'Rock', 'Grass', 'Ice', 'Bug',
       'Bug', 'Psychic', 'Steel', 'Grass', 'Grass', 'Ice', 'Normal',
       'Normal', 'Normal', 'Rock', 'Dark', 'Fire', 'Water', 'Fire',
       'Normal', 'Electric', 'Rock', 'Rock', 'Normal', 'Grass', 'Fire',
       'Normal', 'Grass', 'Water', 'Steel', 'Steel', 'Water', 'Grass',
       'Water', 'Grass', 'Normal', 'Normal', 'Bug', 'Bug', 'Bug', 'Grass',
       'Grass', 'Fire', 'Rock', 'Grass', 'Water', 'Water', 'Bug',
       'Normal', 'Bug', 'Psychic', 'Steel', 'Ghost', 'Normal', 'Fire',
       'Bug', 'Fire', 'Fire', 'Normal', 'Grass', 'Grass', 'Grass',
       

In [8]:
images/=255
images=images.astype('float32')

In [9]:
# integer encode
label_encoder = LabelEncoder()
image_labels = label_encoder.fit_transform(image_labels)
# one hot encode
onehot_encoder = OneHotEncoder(sparse=False)
image_labels = image_labels.reshape(len(image_labels), 1)
image_labels = onehot_encoder.fit_transform(image_labels)
image_labels = np.asarray(image_labels)
print(image_labels)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [10]:
train_data, test_data, train_labels, test_labels=train_test_split(images, image_labels, test_size=0.3, shuffle=True)
train_data, val_data, train_labels, val_labels=train_test_split(train_data, train_labels, test_size=0.1, shuffle=True)

Next we define the model:

In [11]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same', input_shape=(120, 120, 3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(18, activation='softmax'))

adam=tf.keras.optimizers.Adam(lr=10**-4)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 120, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 60, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 58, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 128)       204928    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 256)       2

In [12]:
mc=tf.keras.callbacks.ModelCheckpoint('best_pokemon_model.hdf5', monitor='val_loss', save_best_only=True)

hist=model.fit(train_data, train_labels, batch_size=1, epochs=50, verbose=1, callbacks=[mc], 
               validation_data=(val_data, val_labels))

Train on 509 samples, validate on 57 samples
Epoch 1/50
509/509 [==============================] - 5s 9ms/sample - loss: 2.7860 - accuracy: 0.1375 - val_loss: 2.7736 - val_accuracy: 0.1228
Epoch 2/50
509/509 [==============================] - 3s 6ms/sample - loss: 2.7071 - accuracy: 0.1532 - val_loss: 2.7636 - val_accuracy: 0.1228
Epoch 3/50
509/509 [==============================] - 3s 6ms/sample - loss: 2.6800 - accuracy: 0.1650 - val_loss: 2.7234 - val_accuracy: 0.1754
Epoch 4/50
509/509 [==============================] - 3s 6ms/sample - loss: 2.6089 - accuracy: 0.2063 - val_loss: 2.6348 - val_accuracy: 0.2456
Epoch 5/50
509/509 [==============================] - 3s 6ms/sample - loss: 2.5500 - accuracy: 0.2220 - val_loss: 2.6883 - val_accuracy: 0.1754
Epoch 6/50
509/509 [==============================] - 3s 6ms/sample - loss: 2.5285 - accuracy: 0.2141 - val_loss: 2.6389 - val_accuracy: 0.2807
Epoch 7/50
509/509 [==============================] - 3s 5ms/sample - loss: 2.4996 - accura

In [16]:
model=tf.keras.models.load_model('best_pokemon_model.hdf5')
test_results=model.evaluate(test_data, test_labels, verbose=0)
test_results

[2.5596847926638255, 0.22633745]